In [4]:
import numpy as np
import pandas as pd
import ast
import json
import pickle
import itertools


def personalized_rec(input_ingredients, user_previous_liked):
    df = pd.read_csv('new_df.csv', usecols=['name','recipe_id','minutes','tags','n_steps','steps','ingredients','n_ingredients','rating'])
    df_ingredients = pd.read_csv('ing_rec3.csv', usecols=['ingredient','recipes'])

    def str_to_list(l):
        try:
            return ast.literal_eval(l)
        except ValueError:
            return []

    def create_index(df):
       inverted_index = {} 
       count=0
       for recipe in df['ingredients_list']:
        for ing in recipe:
            if ing not in inverted_index:  
               inverted_index[ing]=[] 
            inverted_index[ing].append(df['recipe_id'][count])
        count+=1
       return inverted_index

    def find_intersection(lists):
        sets = [set(lst) for lst in lists]
        intersection = set.intersection(*sets)
        result = list(intersection)
        return result

    def boolean_retrieval(input_ingredients):
        inverted_index=create_index(df)
        word_doc=[]
        for w in input_ingredients:
            if w in inverted_index:
               word_doc.append(list(inverted_index[w]))
        matching_recipes=find_intersection(word_doc)
        return matching_recipes
    
    def jaccard_similarity(list1, list2):
        set1 = set(list1)
        set2 = set(list2)
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        similarity = len(intersection) / len(union)
        return similarity
    def item_item(each_liked_id):
        ingredients_dict = {}
        for i, ingredient in enumerate(df_ingredients['ingredient']):
            ingredients_dict[ingredient] = i
        recipe_list = df['recipe_id'].tolist()
        recipe_list = sorted(recipe_list)
        with open('recipe_cosine_sim.pickle', 'rb') as f:
            recipe_sim_arr = pickle.load(f)
            recipe_sim_arr = np.array(recipe_sim_arr)
        item_ans=[]
        similar_recipes=recipe_sim_arr[recipe_to_id[each_liked_id]]
        ascending_indices = np.argsort(similar_recipes)
        descending_indices = ascending_indices[::-1]
        count=0
        for i in descending_indices:
            item_ans.append(id_to_recipe[i])
            count+=1
            if count==10:
                break
        return item_ans
    
    
    df['ingredients_list'] = df['ingredients'].apply(str_to_list)
    df['tags_list'] = df['tags'].apply(str_to_list)
    df['steps_list'] = df['steps'].apply(str_to_list)
    recipe_ids=boolean_retrieval(input_ingredients)
    # item-item
    all_items=[]
    df_sorted = df.sort_values(by=['rating'], ascending=False)
    df_sorted = df_sorted.reset_index(drop=True)
    df_sorted['recipe_id_name'] = df_sorted.index 
    recipe_to_id = dict(zip(df_sorted['recipe_id'], df_sorted['recipe_id_name']))
    id_to_recipe = dict(zip(df_sorted['recipe_id_name'],df_sorted['recipe_id']))    
    for each_liked_id in user_previous_liked:  
             if each_liked_id in recipe_ids:
                item_ans=item_item(each_liked_id)
                if len(all_items) == 0:
                      all_items.append(item_ans)
                else:
                    all_items.extend(item_ans)
    # all_items=[x for x in all_items if x != []]
    all_items = list(itertools.chain(*all_items))
    selected_rows = df.loc[df['recipe_id'].isin(all_items)]
    subset_df = df[df['recipe_id'].isin(recipe_ids)]
    subset_df['similarity'] = subset_df['ingredients_list'].apply(lambda x: jaccard_similarity(x, input_ingredients))
    subset_df = subset_df.sort_values(by='similarity',ascending=False)
    tags_list = []
    for lst in subset_df['tags_list'].head(5):
     tags_list.extend(lst)
    subset_df['similarity_tags'] = subset_df['tags_list'].apply(lambda x: jaccard_similarity(x,tags_list))
    subset_df = subset_df.sort_values(by='similarity_tags',ascending=False)
    ans = {}
    subset_df['tags'] = subset_df['tags_list']
    subset_df['steps'] = subset_df['steps_list']
    selected_rows['tags'] = selected_rows['tags_list']
    selected_rows['steps'] = selected_rows['steps_list']
    ob1 = subset_df[0:5].to_dict(orient='records')
    ob2 = selected_rows[0:5].to_dict(orient='records')
    ans['res'] = ob1+ob2
    return ans

In [6]:
input_ingredients = ['tomatoes', 'onions', 'garlic']  # example input from user
user_previous_liked=[999,2647,5216,657,10000,6985,203,645,9984,9841,1290,9626]
personalized_rec(input_ingredients,user_previous_liked)

/var/folders/cj/rhxllj854kxg8fg68tz3_hpc0000gn/T/ipykernel_11918/937640708.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['similarity'] = subset_df['ingredients_list'].apply(lambda x: jaccard_similarity(x, input_ingredients))


{'res': [{'name': 'vegetable medley casserole',
   'recipe_id': 61226,
   'minutes': 80,
   'tags': ['weeknight',
    'time-to-make',
    'course',
    'main-ingredient',
    'preparation',
    'occasion',
    'low-protein',
    'healthy',
    'casseroles',
    'main-dish',
    'side-dishes',
    'vegetables',
    'oven',
    'potluck',
    'low-fat',
    'vegan',
    'vegetarian',
    'stove-top',
    'dietary',
    'low-sodium',
    'low-cholesterol',
    'low-saturated-fat',
    'low-calorie',
    'low-carb',
    'healthy-2',
    'low-in-something',
    'cauliflower',
    'green-yellow-beans',
    'onions',
    'peppers',
    'tomatoes',
    'taste-mood',
    'to-go',
    'equipment',
    'presentation',
    'served-hot',
    '4-hours-or-less'],
   'n_steps': 15,
   'steps': ['preheat your oven to 350f degrees',
    'fill a large saucepan with one to two inches of water , and add your potatoes , carrots , cauliflowerets , and green beans',
    'bring to a boil over high heat , reduc